# Oxylabs Search


> [Oxylabs Search](https://github.com/oxylabs/oxylabs-sdk-python) TODO: Desciption 


## Setup
Following the [instruction](https://developers.oxylabs.io/scraper-apis/web-scraper-api/google#getting-started) TODO: Register instructions

The integration lives in the `langchain-community` package.

In [ ]:
# %pip install -U langchain-community

In [1]:
import getpass
import os
from pprint import pprint

os.environ["OXYLABS_USERNAME"] = getpass.getpass()
os.environ["OXYLABS_PASSWORD"] = getpass.getpass()

In [ ]:
from langchain_community.utilities import OxylabsSearchAPIWrapper


In [ ]:
search = OxylabsSearchAPIWrapper()


In [ ]:
pprint(search.run("python"), indent=4)

## Number of results
You can use parameters "start_page",  "pages" and "limit" to set the number of results

In [ ]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 5,
        "limit": 30
    }
)


In [ ]:
pprint(search.run("python"), indent=4)

## Metadata Results

In [ ]:
search = OxylabsSearchAPIWrapper()


In [ ]:
search.results("apples")


In [ ]:
## Tool Usage

In [4]:
import os

from langchain_community.tools.oxylabs_search import OxylabsSearchResults, OxylabsSearchRun
from langchain_community.utilities import OxylabsSearchAPIWrapper

api_wrapper = OxylabsSearchAPIWrapper()


In [ ]:
tool_results = OxylabsSearchResults(wrapper=api_wrapper)
tool_results

In [ ]:
tool_run = OxylabsSearchRun(wrapper=api_wrapper)
tool_run

In [ ]:
import json

# .invoke wraps utility.results
response_results = tool_results.invoke("What is the weather in Shanghai?")
response_results = json.loads(response_results)
for item in response_results:
    print(item)

In [ ]:
# .invoke wraps utility.results
response_run = tool_run.invoke("What is the weather in Shanghai?")

pprint(response_run)

## Chaining


In [ ]:
# Install the required libraries
%pip install --upgrade --quiet langchain langchain-openai langchainhub langchain-community

In [5]:
# Import necessary modules
import getpass
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI  # Use OpenAIChat instead of AzureChatOpenAI

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Define assistant instructions and pull a base prompt template
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize OpenAI chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Define tools and agent setup
tool = OxylabsSearchRun(wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Set up and invoke the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest Burning Man floods?"})

/home/karolisk/workspace/langchain/.venv/lib/python3.10/site-packages/langsmith/client.py:351: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'latest Burning Man floods news'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331
    DESC: Sep 4, 2023 — Tens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend.
    TITLE: Burning Man flooding: What happened to stranded ...
    URL_SHOWN: https://abcnews.go.com› story
    POS_OVERALL: 1
    FAVICON_TEXT: ABC News - Breaking News, Latest News and Videos
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://www.foxweather.com/weather-news/burning-man-festivalgoers-experience-travel-issues-black-rock-city-nevada-rain
    DESC: Aug 24, 2024 — Monsoon rains on Friday and over the weekend caused flash flooding at Burning Man, turning the usually dry, desert landscape of the festival ...
    TITLE: Travel to Burning Man delayed due to thund

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In the latest Burning Man floods, tens of thousands of attendees were stranded at the festival due to a downpour and massive flooding. The flooding occurred over the weekend, causing travel delays and challenges for festival-goers. Authorities were investigating one death related to the flooding. Attendees at the countercultural music and arts event faced difficulties due to the unexpected weather conditions. If you would like more details or specific information, feel free to ask.

> Finished chain.


{'input': 'What happened in the latest Burning Man floods?',
 'output': 'In the latest Burning Man floods, tens of thousands of attendees were stranded at the festival due to a downpour and massive flooding. The flooding occurred over the weekend, causing travel delays and challenges for festival-goers. Authorities were investigating one death related to the flooding. Attendees at the countercultural music and arts event faced difficulties due to the unexpected weather conditions. If you would like more details or specific information, feel free to ask.'}

In [8]:
agent_executor.invoke({"input": "Who won latest 2024 Lithuanian elections and why, please make an essay from search results."})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': '2024 Lithuanian elections winner'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://apnews.com/article/lithuania-parliamentary-election-new-government-blinkeviit-e7f3344f8e15c07f9a0d46d7f4cb9096
    DESC: 11 hours ago — Lithuania's center-left opposition parties celebrated victory after prevailing over the center-right ruling coalition in the final round of ...
    TITLE: Social Democrats win Lithuania's election, overcoming ...
    URL_SHOWN: https://apnews.com› article
    POS_OVERALL: 2
    FAVICON_TEXT: AP News
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://www.reuters.com/world/europe/lithuania-opposition-seeks-cement-election-win-run-off-votes-2024-10-27/
    DESC: 1 day ago — The opposition Social Democrats claimed victory in Lithuania's parliamentary election on Sunday, which was dominated by frustration with the ...
    TITLE: Lithuania leftist opposition win election, eyes coalition
    URL_SHOWN: htt

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In the latest 2024 Lithuanian elections, the center-left opposition parties, specifically the Social Democrats, emerged victorious over the center-right ruling coalition. The victory was celebrated as the Social Democrats prevailed in the final round of the parliamentary election. The election results were dominated by frustration with the existing government, leading to a significant shift in power.

The Social Democrats claimed victory in Lithuania's parliamentary election, as reported by reputable sources such as [AP News](https://apnews.com/article/lithuania-parliamentary-election-new-government-blinkeviit-e7f3344f8e15c07f9a0d46d7f4cb9096), [Reuters](https://www.reuters.com/world/europe/lithuania-opposition-seeks-cement-election-win-run-off-votes-2024-10-27/), and [The Guardian](https://www.theguardian.com/world/2024/oct/28/lithuania-social-democratic-party-election-winner-bolster-defence-russia-threat). The election outcome signifies a shift in the political landscape of Lithuania

{'input': 'Who won latest 2024 Lithuanian elections and why, please make an essay from search results.',
 'output': "In the latest 2024 Lithuanian elections, the center-left opposition parties, specifically the Social Democrats, emerged victorious over the center-right ruling coalition. The victory was celebrated as the Social Democrats prevailed in the final round of the parliamentary election. The election results were dominated by frustration with the existing government, leading to a significant shift in power.\n\nThe Social Democrats claimed victory in Lithuania's parliamentary election, as reported by reputable sources such as [AP News](https://apnews.com/article/lithuania-parliamentary-election-new-government-blinkeviit-e7f3344f8e15c07f9a0d46d7f4cb9096), [Reuters](https://www.reuters.com/world/europe/lithuania-opposition-seeks-cement-election-win-run-off-votes-2024-10-27/), and [The Guardian](https://www.theguardian.com/world/2024/oct/28/lithuania-social-democratic-party-electio

In [7]:
agent_executor.invoke({"input": "What is the most profitable company in lithuania in 2024?"})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'most profitable company in Lithuania 2024'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://ceoworld.biz/2024/02/19/lithuanias-largest-companies-by-market-capitalization-2024/
    DESC: Feb 19, 2024 — As of October 24, 2024, AB IGNITIS GRUPE is the most valuable company in Lithuania, with a market cap of $1.51 billion. Following it are TELIA ...
    TITLE: Lithuania's Largest Companies by market capitalization, ...
    URL_SHOWN: https://ceoworld.biz › Latest › Banking and Finance
    POS_OVERALL: 1
    FAVICON_TEXT: CEOWORLD magazine
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://www.hithorizons.com/eu/analyses/country-statistics/lithuania
    DESC: Lithuania: TOP 10 Companies with Highest Sales 194,288 registered companies ; 1. ORLEN LIETUVA AB € 4.264B ; 2. MAXIMA LT UAB € 1.985B ; 3. THERMO FISHER ...
    TITLE: Industry Breakdown of Companies in Lithuania
    URL_SHOWN: https://www.hithorizons.com › analy

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE, with a market capitalization of $1.51 billion.

> Finished chain.


{'input': 'What is the most profitable company in lithuania in 2024?',
 'output': 'The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE, with a market capitalization of $1.51 billion.'}